# Standardize and deduplicate fiction metadata

This notebook begins after masterficmeta has been created.

The next step is to produce a dataset where there's a single copy of each physical volume. In terms of [FRBR,](https://en.wikipedia.org/wiki/Functional_Requirements_for_Bibliographic_Records) this is roughly the "manifestation" level.

In other words, after this round of dedup, there will still be several copies of *Middlemarch* in the dataset: maybe a three-volume edition from the 1870s, as well as one-volume editions in 1892 and 1950, adding up to five volumes. But there should be only one copy of each volume, in each edition.

I'll achieve that very simply by relying on HTRC record IDs + volume numbers.

This notebook also prepares for the next stage of deduplication by loosely standardizing author names.

The output is written to disk as manifestationmeta.tsv.


In [2]:
# Some generic imports.

import csv
import pandas as pd

### General strategy for standardizing authors

We're not going to use fuzzy matching on this pass; we're going to make very conservative assumptions and link authors only if

    1) They have the same last name
    2) Also the same initials
    3) And also *either* the same birthdate, or
    4) Are listed as authoring the same title.

Very short author names will also be ignored.

To do this we need dictionaries of authors' birthdates and titles-authored. While we're at it we can group authors into blocks that share the same first two letters, to speed up the comparison process later.

In [3]:
meta = pd.read_csv('../masterficmetadata.tsv', sep = '\t', low_memory = False)
grouped = meta.groupby('author')
titlesets = dict()
birthdates = dict()
authorblocks = dict()

for author, group in grouped:
    birthset = set()
    titleset = set()
    for ix in group.index:
        authordate = group.loc[ix, 'authordate']
        if not pd.isnull(authordate) and len(authordate) >= 4:
            try:
                birth = int(authordate[0:4])
                birthset.add(birth)
            except:
                pass
        title = group.loc[ix, 'shorttitle']
        if not pd.isnull(title):
            if len(title) > 20:
                title = title[0:20]
            titleset.add(title)
    if len(birthset) == 1:
        birthdates[author] = birthset.pop()
    titlesets[author] = titleset
    
    if len(author) > 2:
        initial = author[0:2]
    else:
        initial = 'xx'
    if initial not in authorblocks:
        authorblocks[initial] = set()
    authorblocks[initial].add(author)
        

#### actually generate standardized names

We go through each block, looking for pairs of authors who meet the above criteria.

When a matched pair are found, we add them as a new group, unless one member of the pair is in an existing group, in which case, they are both added to that group.

In [4]:
groups = []

def zapnsplit(a):
    a = a.replace('from old catalog', ' ')
    a = a.replace('.', ' ')
    a = a.replace('(', ' ')
    a = a.replace(')', ' ')
    a = a.replace('[', ' ')
    a = a.replace(']', ' ')
    a = a.replace(',', ' ')
    a = a.replace('Mrs', ' ')
    a = a.replace('Sir', ' ')
    # we don't want honorifics to interfere with 
    # identity
    
    parts = a.lower().split()
    lastname = parts[0]
    parts = [x for x in parts if len(x) > 0]
    return set(parts), lastname

def matchnames(namea, nameb):
    a, lasta = zapnsplit(namea)
    b, lastb = zapnsplit(nameb)
    
    if lasta != lastb:
        return False
    
    missing = a - b
    for m in missing:
        if m[0] not in b:
            return False
    # for each word that's missing in b,
    # we check to see if the initial is present in B
    
    missing = b - a
    for m in missing:
        if m[0] not in a:
            return False
    # likewise for a
    
    # If all tests are passed,  
    return True

# iterate through author blocks and actually do the work

for initial, block in authorblocks.items():
        
    for a1 in block:
        for a2 in block:
            if a1 == a2:
                continue
            if len(a1) < 9 or len(a2) < 9:
                continue
            if a1[0:5] != a2[0:5]:
                continue
            
            if len(a1) > 21:
                trunca1 = a1[0:21]
            else:
                trunca1 = a1
                
            if len(a2) > 21:
                trunca2 = a2[0:21]
            else:
                trunca2 = a2
                
            if trunca1 != trunca2 and not matchnames(a1, a2):
                continue
            
            titlematch = 0
            
            birthmatch = False
            if (a1 in birthdates and a2 in birthdates) and (birthdates[a1] == birthdates[a2]):
                birthmatch = True
                titlematch += 1
                # not requiring titlematch if birthdates match
                
            elif a1 not in birthdates:
                birthmatch = True
            elif a2 not in birthdates:
                birthmatch = True
            
            if not birthmatch:
                continue
            
            for t1 in titlesets[a1]:
                for t2 in titlesets[a2]:
                    if t1 == t2:
                        titlematch += 1
            
            if titlematch == 0:
                continue
            else:
                found = False
                for g in groups:
                    if a1 in g or a2 in g:
                        g.add(a1)
                        g.add(a2)
                        found = True
                        break
                        
                if not found:
                    groups.append({a1, a2})
                    
                

In [5]:
len(groups)

1157

In [6]:
# Write the author groups to file for future reference.

with open('authorgroups.tsv', mode = 'w', encoding = 'utf-8') as f:
    for g in groups:
        of_record = ''
        for n in g:
            official = n.replace('from old catalog', '')
            official = official.strip('[], .')
            if official[-1].isupper():
                official = official + '.'
            if len(of_record) < 1 or len(official) > len(of_record):
                of_record = official
        outline = of_record + '\t' + '\t'.join(g) + '\n'
        f.write(outline)
        

Now let's read that back in, to create a translation dictionary.

In [7]:
author_trans = dict()

with open('authorgroups.tsv', encoding = 'utf-8') as f:
    for line in f:
        fields = line.strip('\n').split('\t')
        official = fields [0]
        for name in fields:
            author_trans[name] = official

Actually standardize the author names.

In [8]:
for idx in meta.index:
    name = meta.loc[idx, 'author']
    if name in author_trans:
        meta.loc[idx, 'author'] = author_trans[name]

#### Let's also standardize authordates

Some rows have birth and/or death dates for an author; others for the same author may lack that info. We're going to want to take the richest available info, and spread it across all rows for the author. "Longest" is not the world's best metric of "richest," but in practice, for this case, it will do.

In [9]:
authordates = dict()
grouped = meta.groupby('author')
for author, group in grouped:
    longest = ''
    for d in group.authordate:
        d = str(d)
        if pd.isnull(d):
            continue
        if len(d) > len(longest) and d.lower() != 'nan':
            longest = d
    authordates[author] = longest


### The actual deduplication

We create a column that is str(recordid) + str(volnum), so we can group on the combination. (Passing both to groupby gives you a multi-index, which is more complexity than I need today.)

We also record the number of instances of a recordid-volnum combination that are being collapsed.

In [13]:
meta['groupingcolumn'] = ''
for idx in meta.index:
    record = str(meta.loc[idx, 'recordid'])
    volnum = meta.loc[idx, 'volnum']
    if pd.isnull(volnum):
        volnum = 'nan'
    else:
        volnum = str(volnum)
    meta.loc[idx, 'groupingcolumn'] = record + '+' + volnum
    

1


KeyError: 'the label [docid] is not in the [index]'

In [16]:
meta.reset_index(inplace = True)
kept = []
ctr = 0
instances = dict()
grouped = meta.groupby(['groupingcolumn'])
for key, group in grouped:
    ctr += 1
    if ctr % 5000 == 1:
        print(ctr)
    keeper = ''
    lowest = 2100
    for idx in group.index:
        date = int(group.loc[idx, 'inferreddate'])
        if (date < lowest and date > 1699) or lowest == 2100:
            lowest = date
            
            if lowest < 1700:
                lowest = 2100
            # dubious "dates" should not outcompete real dates
            
            keeper = group.loc[idx, 'docid']
            if type(keeper) != str:
                keeper = keeper[0]
    numcopies = len(group.index)
    instances[keeper] = numcopies
    kept.append(keeper)

1
5001
10001
15001
20001
25001
30001
35001
40001
45001
50001
55001
60001
65001
70001
75001
80001
85001
90001
95001
100001
105001
110001
115001
120001
125001
130001
135001
140001
145001
150001
155001
160001
165001
170001
175001


That created a list of docids to keep. Now we just have to keep them.

In [18]:
# actually do the deduplication
print(meta.shape)
meta.set_index('docid', inplace = True)
deduped = meta.loc[kept, : ]
print(deduped.shape)
deduped = deduped[~deduped.index.duplicated(keep='first')]
print(deduped.shape)

(210278, 26)
(176623, 25)
(176623, 25)


In winnowing duplicates, we don't want to lose authors' birth and death dates. Let's insure we have the richest information.

Also, while we're at it, let's add a column recording the number of instances for each record-vol.

In [24]:
def map_instances(docid):
    global instances
    if docid in instances:
        return instances[docid]
    else:
        return 0

deduped['instances'] = deduped.index.map(map_instances)

for idx in deduped.index:
    authdate = deduped.loc[idx, 'authordate']
    if pd.isnull(authdate):
        authdate = ''
    author = deduped.loc[idx, 'author']
    if author in authordates and len(authordates[author]) > len(authdate) and authordates[author] != 'nan':
        deduped.loc[idx, 'authordate'] = authordates[author]

In [25]:
deduped.head()

,index,oldauthor,author,authordate,inferreddate,latestcomp,datetype,startdate,enddate,imprint,...,oclc,place,recordid,enumcron,volnum,title,parttitle,shorttitle,groupingcolumn,instances
docid,,,,,,,,,,,,,,,,,,,,,
uc2.ark:/13960/t4mk67z1j,60095,"Jones, J. D. (John Daniel)","Jones, J. D. (John Daniel)",1865-1942.,1900,1900,,1900,1989,New York|George H. Doran|19--,...,NaN,nyu,100000247,NaN,NaN,The Lord of life and death / | $c: by J.D. Jones.,NaN,The Lord of life and death,100000247+nan,1
uc2.ark:/13960/t39z92s0m,60096,"Ovcharenko, Ivan Vasilʹevich","Ovcharenko, Ivan Vasilʹevich",NaN,1900,1900,,1900,1985,London|Modern Books|19--?,...,NaN,enk,100000271,NaN,NaN,In a ring of fire; | memories of a partisan.,NaN,In a ring of fire; memories of a partisan,100000271+nan,1
uc2.ark:/13960/t0ht2j50b,60097,"Phillpotts, Eden","Phillpotts, Eden",1862-1960.,1900,1900,,1900,1987,Paris|Thomas Nelson and Sons|19--?,...,NaN,fr,100000276,NaN,NaN,Old Delabole / | $c: by Eden Phllpotts.,NaN,Old Delabole,100000276+nan,1
uc2.ark:/13960/t1sf2p90r,60098,"Rosborough, Alexander J","Rosborough, Alexander J",NaN,1900,1900,,1900,1986,"Yreka, CA|News-Journal Print Shop|19]̲̲",...,NaN,cau,100000283,NaN,NaN,The mountie and the sourdough / | $c: by Alexa...,NaN,The mountie and the sourdough,100000283+nan,1
uc2.ark:/13960/t9c53hj4v,60099,"Verne, Jules","Verne, Jules",1828-1905.,1900,1900,,1900,1986,New York|Phoenix Publishing Co.|19--?,...,NaN,nyu,100000299,NaN,NaN,The tour of the world in eighty days / | $c: b...,NaN,The tour of the world in eighty days,100000299+nan,1


Now we just write to file, after dropping an extra column and sorting.

In [26]:
deduped.drop(labels = ['index', 'groupingcolumn'], axis = 1, inplace = True)
deduped.sort_values(by = ['inferreddate', 'recordid', 'volnum'], inplace = True)
deduped.to_csv('manifestationmeta.tsv', sep = '\t')
